In [1]:
import os

In [55]:
import geopandas as gpd
from wildfire.countryFormat import gdf_to_fc
# import numpy as np
# from shapely.geometry import box
import pandas as pd
import folium
from folium.features import GeoJsonTooltip

from shapely.geometry import box

import matplotlib.cm as cm
import matplotlib.colors as mcolors
from branca.colormap import LinearColormap, linear
from matplotlib import colormaps

In [ ]:
# Run this cell after from wildfire.countryFormat import gdf_to_fc
import ee
ee.Authenticate()
ee.Initialize(project='wildfire-monitoring-2025')

In [ ]:
portugal_grid_gdf = gpd.read_file(os.path.join('data', 'geojson', 'portugal_grid.geojson'))
portugal_border_gdf = gpd.read_file(os.path.join('data', 'geojson', 'portugal_border.geojson'))
portugal_dpts_gdf = gpd.read_file(os.path.join('data', 'geojson', 'portugal_dpts.json'))

In [10]:
path = os.path.join('data', 'csv', 'ndvi_portugal.csv')
ndvi_df = pd.read_csv(path)

path = os.path.join('data', 'csv', 'fire_portugal.csv')
fire_df = pd.read_csv(path)

path = os.path.join('data', 'csv', 'sm_portugal.csv')
sm_df = pd.read_csv(path)

path = os.path.join('data', 'csv', 'weather_portugal.csv')
weather_df = pd.read_csv(path)

# Layers

In [59]:
# the grid GeoDataFrame
grid_layer = folium.GeoJson(
    portugal_grid_gdf,
    name='Grid',
    style_function=lambda x: {
        'color': 'black',
        'weight': .5,
        'fillOpacity': 0,
        'opacity': 0.2
    }
)

portugal_border = folium.GeoJson(
    portugal_border_gdf,
    name='Portugal Border',
    style_function=lambda feature: {
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0,
        'opacity': 0.7
    }
)

portugal_dpts = folium.GeoJson(
    portugal_dpts_gdf,
    name='Portugal Departments',
    style_function=lambda feature: {
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0,
        'opacity': 0.7
    },
    tooltip=GeoJsonTooltip(
        fields=["name"],
        aliases=["District:"],
        localize=True,
        sticky=False
    ),
    # popup=folium.GeoJsonPopup(fields=["name"])
)

In [ ]:
# import branca.colormap as cm

ndvi_df_0 = ndvi_df.loc[ndvi_df['date']== '2020-01-01', :].copy()

# Tile size in degrees (20km ~ 0.18)
tile_size = 0.09
ndvi_df_0['geometry'] = ndvi_df_0.apply(
    lambda row: box(
        row['centroid_lon'] - tile_size/2,
        row['centroid_lat'] - tile_size/2,
        row['centroid_lon'] + tile_size/2,
        row['centroid_lat'] + tile_size/2
    ), axis=1
)

# Convert to GeoDataFrame
ndvi_gdf = gpd.GeoDataFrame(ndvi_df_0, geometry='geometry', crs='EPSG:4326')

# Create color scale
min_val = -2000
max_val = 10000#ndvi_df_0['avg_NDVI'].max()

mpl_BrBG = colormaps['BrBG']
BrBG_colors = [mcolors.rgb2hex(mpl_BrBG(i)) for i in range(mpl_BrBG.N)]

mpl_RdYlGn = colormaps['RdYlGn']
RdYlGn_colors = [mcolors.rgb2hex(mpl_RdYlGn(i)) for i in range(mpl_RdYlGn.N)]

NDVI_colormap = LinearColormap(RdYlGn_colors, vmin=min_val, vmax=max_val)
NDVI_colormap.caption = 'Normalized Difference Vegetation Index (NDVI)'

# Define a style function for GeoJson
def style_function(feature):
    value = feature['properties']['avg_NDVI']
    color = NDVI_colormap(value) if value>0 else 'rgba(0, 0, 0, 0)'
    return {
        'fillColor': color,
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.75,
        'opacity': 0.,         # Line opacity
    }

# Add GeoJson with tiles
portugal_ndvi = folium.GeoJson(
    ndvi_gdf,
    name='avg_NDVI',
    style_function=style_function,
    tooltip=folium.features.GeoJsonTooltip(fields=['tile_id', 'date', 'avg_NDVI'])
)

# Plot

In [65]:
# 2. Create a Folium map centered on Portugal
center_lat = 38.736946 #(ymin + ymax) / 2
center_lon = -9.142685 #(xmin + xmax) / 2

tiles_option = 'OpenStreetMap' # Other options: 'OpenStreetMap', 'Stamen Toner', 'Stamen Terrain', 'Stamen Watercolor', 'CartoDB positron', 'CartoDB dark_matter'
m = folium.Map(location=[center_lat, center_lon], zoom_start=7, max_bounds = True)
# folium.TileLayer('OpenStreetMap').add_to(m)
# folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('Stamen Toner').add_to(m)
# folium.TileLayer('CartoDB positron').add_to(m)
# folium.TileLayer('CartoDB dark_matter').add_to(m)


grid_layer.add_to(m)
portugal_ndvi.add_to(m)
# portugal_border.add_to(m)
portugal_dpts.add_to(m)
NDVI_colormap.add_to(m)
# 4. Show map
folium.LayerControl().add_to(m)
m
# m.save(os.path.join('data', 'maps_html', 'ndvi_portugal.html'))